# DESCARGA DE NOTICIAS DE GOOGLE (GOOGLE NEWS)

Este notebook describe el paso a paso que permitirá descargar las noticias que aparecen en Google

### Paso 1: Instalación de software necesario

--- Python

Puede obtenerse instalando [Anaconda](https://www.anaconda.com). Para ello, ingrese [aquí](https://www.anaconda.com/products/distribution), descargue el instalador y ejecútelo.

--- Git

Ingrese [aquí](https://git-scm.com/downloads), descargue el instalador y ejecútelo.

### Paso 2: Instalación de dependencias

Para ello, ejecute la siguiente celda

In [ ]:
!pip install scrapy==2.6.1
!pip install scrapy-splash==0.8.0
!pip install pyodbc==4.0.32
!pip install pysftp==0.2.9
!pip install ipywidgets==7.6.5
!pip install html2text==2020.1.16

### Paso 2.1: Carga de librerías necesarias

Para ello, ejecute la siguiente celda

In [1]:
from IPython.display import clear_output
import pandas as pd
from toolbox import get_date_inputs, upload_to_lz, clean_news, save_local

print('LIBRERÍAS IMPORTADAS CORRECTAMENTE')

LIBRERÍAS IMPORTADAS CORRECTAMENTE


### Paso 3: Descarga de la última versión del código fuente (código que permitirá la descarga de las noticias)

Para ello, es necesario ejecutar la siguiente celda

In [4]:
!if exist google_news_scraper (rmdir /s /q google_news_scraper)

REPOSITORY_URL = 'https://github.com/daneelfx/google_news_scraper'

!git clone $REPOSITORY_URL
clear_output(wait = True)

!if exist google_news_scraper (echo 'LA ULTIMA VERSION DEL CODIGO FUE DESCARGADA EXITOSAMENTE') else (echo '¡ERROR!: EL CODIGO NO PUDO SER DESCARGADO. REVISE CONEXION Y VPN')

'�ERROR!: EL CODIGO NO PUDO SER DESCARGADO. REVISE CONEXION Y VPN'


### Paso 4: Descarga de las noticias de interés

Las noticias se pueden filtrar por:


* *Términos de búsqueda*  

Inmediatamente después de la siguiente celda usted tendrá la oportunidad de ingresar los términos sobre los cuales quiere buscar noticias. Cada vez que le aparezca
un campo para ingresar texto escriba el término de interés y presione la tecla *enter*. Por favor no escriba palabras que no agregan significado; por ejemplo, en vez de escribir "Banco de la República" escriba "Banco República", o en vez de "Grupo de Energía de Bogotá" escriba "Grupo Energía Bogotá". Si ya escribió todos los términos de interés, en vez de escribir uno adicional presione la tecla *enter*.
En caso de que quiera buscar todas las noticias (es decir, todas y absolutamente todas las que el programa encuentre), simplemente presione la tecla *enter* cuando le aparezca por primera vez el campo de texto.

In [ ]:
SEARCH_TERMS = []

while True:
  search_term = input('Ingrese el término de búsqueda o presione la tecla ENTER para terminar\n').strip().upper()
  if not search_term:
    break
  SEARCH_TERMS.append(search_term)

print(f'Términos de búsqueda: {SEARCH_TERMS}' if SEARCH_TERMS else 'LOS TÉRMINOS DE BÚSQUEDA NO FUERON INGRESADOS')

* *Lenguaje de búsqueda*  

Pueden obtenerse resultados en inglés o en español

In [ ]:
language = 'en' if input('Ingrese el idioma de búsqueda ("es" para español y "en" para inglés)').strip().lower() == 'en' else 'es'
language

* *Fechas inicial y final*

De la misma manera, inmediatamente después de siguiente celda debe indicar el periodo en el que se quieren buscar noticias. Para ello, ingrese el mes (por ejemplo: "Febrero") y el año (por ejemplo: 2021) inicial y el mes y el año final.

In [ ]:
start_day, start_month, start_year, end_day, end_month, end_year = get_date_inputs()

In [ ]:
SEARCH_TERMS = ['GRUPO ARGOS'] #, 'BANCO BOGOTA'] #, 'BANCOLOMBIA', 'CEMENTOS ARGOS', 'CELSIA', 'CORFICOLOMBIANA', 'DAVIVIENDA', 'ECOPETROL', 'GRUPO AVAL', 'GRUPO ENERGIA BOGOTA', 'ISA', 'NUTRESA', 'GRUPO SURA']
language = 'es'
start_day, start_month, start_year, end_day, end_month, end_year = 1, 1, 2022, 1, 1, 2022

In [ ]:
!if exist news (rmdir /s /q news)
SEARCH_TERMS_STR = ','.join(SEARCH_TERMS)
current_year = int(start_year)
end_year = int(end_year)

while current_year <= end_year:
  current_start_date = f'{current_year}-{start_month if current_year == int(start_year) else "01"}-{start_day if current_year == int(start_year) else "01"}'
  current_end_date = f'{current_year}-{12 if current_year < end_year else end_month}-{31 if current_year < end_year else end_day}'
  !cd google_news_scraper && scrapy crawl news -a search_terms="$SEARCH_TERMS_STR" -a start_date=$current_start_date -a end_date=$current_end_date -a language=$language -o ..\news\news-"$current_year".csv
  # clear_output(wait = True)
  current_year += 1

# clear_output(wait = False)
!if exist news (echo 'NOTICIAS DESCARGADAS CON EXITO') else (echo '¡ERROR!: ALGO OCURRIO, INTENTE NUEVAMENTE ')

### Paso 5: Limpieza de texto y filtrado de las noticias

Las noticias obtenidas tienen algunos caracteres y textos no deseados. Adicionalmente, algunos resultados están duplicados, así como otros que no coinciden con el término de busqueda empleado en ésta. Para limpiar el texto y filtrar las noticias, ejecute la celda inferior.

In [ ]:
for current_year in range(int(start_year), int(end_year) + 1):
  current_news = pd.read_csv(f'news/news-{current_year}.csv')
  current_news = current_news.astype(str)
  
  news_cleaned = clean_news(current_news)
  !del news\news-"$current_year".csv
  news_cleaned.to_csv(f'news/news-{current_year}.csv', index = False)
  print(f'NOTICIAS DEL AÑO {current_year} FUERON LIMPIADAS Y FILTRADAS')

print('*' * 50)
print('LIMPIEZA Y FILTRADO CONCLUYERON EXITOSAMENTE')

### Paso 5.1 (OPCIONAL): Persistencia de las noticias (destino: local)
Las noticias serán guardadas en el mismo directorio de este notebook como **ALL_NEWS.csv**

In [ ]:
save_local('news', 'ALL_NEWS')

### Paso 5.2 (OPCIONAL): Persistencia de las noticias (destino: nube)
En caso de querer subir información a la nube se debe utilizar el método **upload_to_lz()**, en donde *database* es el nombre de la base de datos a la cual usted esté autorizado y *table_name* el nombre con el que la tabla quedará cargada en la nube. Tenga en cuenta que en el mismo directorio de este notebook debe existir un archivo llamado **credentials.json**, debe contener lo siguiente: **{ "user": "su_usuario", "password": "su_contraseña", "server": "sbmdeblze004" }**

In [ ]:
save_local('news', 'ALL_NEWS')
all_news = pd.read_csv('ALL_NEWS.csv')
all_news = all_news.astype(str)

database = 'proceso'
table_name = 'dasolano_google_news'

upload_to_lz(dataframe = all_news, database = database, table_name = table_name)
clear_output(wait = True)
!del ALL_NEWS.csv
print(f'LAS NOTICIAS HAN SIDO CARGADAS A LA NUBE EXITOSAMENTE. UBICACIÓN LZ: {database}.{table_name}')